In [ ]:
import glob
import shutil
import os
import random
import cv2

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import scikitplot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from keras.utils import np_utils

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename));

In [ ]:
path_to_data = '/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/train.csv'

In [ ]:
def load_and_EDA(path_to_data):
    emotion_cat = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}
    
    data = pd.read_csv(path_to_data)
    print(data.head())
    
    temp = data['emotion'].apply(lambda x: emotion_cat[x])
    
    temp.value_counts().plot(kind='bar', figsize=(7, 6), rot=0)
    plt.xlabel("Emotions", labelpad=14)
    plt.ylabel("Count of Emotions", labelpad=14)
    plt.title("Count of the distribution of emotions", y=1.02)
    
    return data
    

In [ ]:
def visualize_data(data, number_of_images_by_class):
    
    '''This function converts pixels to images.
       You need to provide the data frame and the number of images you which to print per class.'''
    
    counter = 0
    emotion_cat = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}
    emotion_num = sorted(data['emotion'].unique())

    plt.gcf().set_size_inches(24, 12)

    for e in emotion_num:
        for j in range(7):
            px = data[data.emotion==e].pixels.iloc[counter]
            px = np.array(px.split(' ')).reshape(48, 48).astype('float32')

            counter += 1
            ax = plt.subplot(7, 7, counter)
            ax.imshow(px, cmap='gray')
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_title(emotion_cat[e])
            plt.tight_layout()

In [ ]:
def prepare_data(data):
    img = data['pixels'].apply(lambda x: np.array(x.split(' ')).reshape(48, 48, 1).astype('float32'))
    img = np.stack(img, axis=0)
    
    le = LabelEncoder()
    label = le.fit_transform(data['emotion'])
    label = np_utils.to_categorical(label)
    label.shape
    
    print(f'Image Shape: {img.shape}')
    print(f'Label Shape: {label.shape}')
    
    return img, label

In [ ]:
def build_data_pipelines(batch_size, train_data):
    
    train_augmentor = ImageDataGenerator(
        rescale = 1. / 255,
        rotation_range=25,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest"
    )
    
    train_generator = train_augmentor.flow(
        train_data[0], train_data[1],
        shuffle=True,
        batch_size=batch_size
    )

    return train_generator

In [ ]:
def build_model(nbr_classes):
    model = Sequential()

    '''model.add(Conv2D(64, (3,3), padding='same', input_shape=(48,48,1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (5,5), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))

    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))'''
    
    model.add(Conv2D(128, (5,5), activation = 'relu', padding = 'same', input_shape=(48,48,1)))
    model.add(Conv2D(128, (5,5), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization())
    
    model.add(Conv2D(256, (5,5), activation = 'relu', padding = 'same'))
    model.add(Conv2D(256, (5,5), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.8))
    model.add(BatchNormalization())
    
    model.add(Conv2D(512, (5,5), activation = 'relu', padding = 'same'))
    model.add(Conv2D(512, (5,5), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.8))
    model.add(BatchNormalization())
    
    model.add(Conv2D(1024, (5,5), activation = 'relu', padding = 'same'))
    model.add(Conv2D(1024, (5,5), activation = 'relu', padding = 'same'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.8))
    model.add(BatchNormalization())

    model.add(Flatten())
    
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.8))
    model.add(BatchNormalization())

    model.add(Dense(nbr_classes, activation='softmax'))
    
    return model

In [ ]:
def train(train_data, valid_data, batch_size, epochs, nbr_classes):

    total_train_imgs = len(train_data[0])
    total_val_imgs = len(valid_data[0])

    print(total_train_imgs, total_val_imgs)
    

    train_generator = build_data_pipelines(
        batch_size=batch_size,
        train_data=train_data,
    )

    model = build_model(nbr_classes)

    optimizer = Adam(lr=0.001)

    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    
    print("[Debugging] So far so good")

    h = model.fit(
        train_generator,
        steps_per_epoch=total_train_imgs // batch_size,
        validation_data=(valid_data[0], valid_data[1]),
        epochs=epochs
    )
    

    print("[INFO] Evaluation phase...")

    predictions = model.predict(valid_data[0]).argmax(axis=1)

    my_classification_report = classification_report(np.argmax(valid_data[1], axis=1), predictions)

    print("[INFO] Classification report : ")
    print(my_classification_report)
    
    print("[INFO] Total wrong validation predictions : ")
    print(np.sum(np.argmax(valid_data[1], axis=1) != predictions))

    print("[INFO] Confusion matrix : ")
    scikitplot.metrics.plot_confusion_matrix(np.argmax(valid_data[1], axis=1), predictions, figsize=(7,7))    
    

In [ ]:
df = load_and_EDA(path_to_data)

In [ ]:
visualize_data(df, number_of_images_by_class=10)

In [ ]:
img, label = prepare_data(df)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(img, label, shuffle=True, stratify=label, test_size=0.2, random_state=42)

print(f'X training shape: {X_train.shape}')
print(f'X validation shape: {X_valid.shape}')
print(f'Y training shape: {y_train.shape}')
print(f'Y validation shape: {y_valid.shape}')

valid_data = (X_valid, y_valid) 
train_data = (X_train, y_train)

In [ ]:
train(train_data, valid_data, batch_size=32, epochs=100, nbr_classes=7)